<a href="https://colab.research.google.com/github/salsaimon/proyecto_recursoshumanos/blob/main/GH_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:

import sys
import os

path='/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos'

os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

In [45]:
con=sql.connect('base.db')  ### crea base de datos
cur=con.cursor()  ### para ejecutar consultas en la base de datos

In [46]:

#### Cargar paquetes siempre al inicio
import pandas as pd ### para manejo de datos
import sqlite3 as sql #### para bases de datos sql
import a_funciones as funciones  ###archivo de funciones propias
import sys ## saber ruta de la que carga paquetes

ModuleNotFoundError: No module named 'a_funciones'